<a href="https://colab.research.google.com/github/KaruDevs/AN-LISIS-DE-DATOS-DE-MIGRACI-N-CON-NUMPY-Y-PANDAS/blob/main/ConsolidadoM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries

### Subtask:
Install the libraries required to work with PDF files in Python.


**Reasoning**:
Install the `PyMuPDF` library using pip.



In [1]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.1 MB/s eta 0:00:00


## Import libraries

### Subtask:
Import the installed libraries into the notebook.


**Reasoning**:
Import the `fitz` library to work with PDF documents.



In [2]:
import fitz

## Load the pdf file

### Subtask:
Write code to load the PDF file.


**Reasoning**:
Load the PDF file using fitz.open() and store the document object in a variable.



In [3]:
doc = fitz.open("/content/EVALUACION FINAL M3.pdf")

## Extract text from the pdf

### Subtask:
Write code to extract text content from the loaded PDF.


**Reasoning**:
Extract text from each page of the PDF and store it in a string variable.



In [4]:
text = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text += page.get_text()

print(text[:500]) # Print the first 500 characters to verify

 
 
EVALUACIÓN FINAL: ANÁLISIS DE DATOS DE MIGRACIÓN CON NUMPY Y PANDAS 
Un instituto de investigación internacional está analizando datos de migración del siglo XXI para 
identificar tendencias y factores socioeconómicos que afectan los movimientos de población. Se ha 
entregado un dataset en formato CSV con información sobre migraciones entre distintos países, 
incluyendo cantidad de migrantes, razones de migración y variables económicas. 
El instituto necesita que analices, transformes y prep


## Provide explanation

### Subtask:
Add markdown cells to explain the code and the process for students.


**Reasoning**:
Add markdown cells to explain the code and the process for students.



In [5]:
%%markdown
# Extracting Text from PDF using Python

This guide explains how to extract text content from a PDF file using the `PyMuPDF` library in Python.

## Purpose

The purpose of this script is to demonstrate a simple and effective way to programmatically access and retrieve the textual information embedded within a PDF document. This can be useful for various applications, such as:

*   Analyzing document content
*   Creating text-based search indexes
*   Converting PDFs to other text formats
*   Automating data extraction from reports or forms

## Process and Code Explanation

The process involves the following steps:

1.  **Installing the necessary library:** We use `PyMuPDF`, a Python binding for MuPDF, which is a lightweight PDF, XPS, and E-book interpreter library.
    ```bash
    %pip install PyMuPDF
    ```
    This command installs the `PyMuPDF` package and its dependencies into your Python environment, making the `fitz` module available for use.

2.  **Importing the library:** We import the `fitz` module, which is the primary interface for interacting with PDF documents using `PyMuPDF`.
    ```python
    import fitz
    ```

3.  **Loading the PDF file:** We open the target PDF file using `fitz.open()`.
    ```python
    doc = fitz.open("/content/EVALUACION FINAL M3.pdf")
    ```
    This line opens the specified PDF file and creates a `Document` object (`doc`) that represents the PDF. This object allows us to access its properties and content.

4.  **Iterating through pages and extracting text:** We loop through each page of the document and extract its text content.
    ```python
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()
    ```
    - `text = ""`: Initializes an empty string variable to store the extracted text from all pages.
    - `for page_num in range(doc.page_count):`: This loop iterates through each page of the document. `doc.page_count` gives the total number of pages in the PDF.
    - `page = doc.load_page(page_num)`: Loads the current page object based on its index (`page_num`).
    - `text += page.get_text()`: Extracts the text content from the current page using the `get_text()` method and appends it to the `text` variable.

5.  **Verifying the extraction (Optional):** We can print a portion of the extracted text to confirm that the process was successful.
    ```python
    print(text[:500]) # Print the first 500 characters to verify
    ```
    This line prints the first 500 characters of the `text` variable, giving us a glimpse of the extracted content to ensure it looks as expected.

This process allows you to programmatically extract the entire text content of a PDF document for further processing or analysis.


# Extracting Text from PDF using Python

This guide explains how to extract text content from a PDF file using the `PyMuPDF` library in Python.

## Purpose

The purpose of this script is to demonstrate a simple and effective way to programmatically access and retrieve the textual information embedded within a PDF document. This can be useful for various applications, such as:

*   Analyzing document content
*   Creating text-based search indexes
*   Converting PDFs to other text formats
*   Automating data extraction from reports or forms

## Process and Code Explanation

The process involves the following steps:

1.  **Installing the necessary library:** We use `PyMuPDF`, a Python binding for MuPDF, which is a lightweight PDF, XPS, and E-book interpreter library.
    ```bash
    %pip install PyMuPDF
    ```
    This command installs the `PyMuPDF` package and its dependencies into your Python environment, making the `fitz` module available for use.

2.  **Importing the library:** We import the `fitz` module, which is the primary interface for interacting with PDF documents using `PyMuPDF`.
    ```python
    import fitz
    ```

3.  **Loading the PDF file:** We open the target PDF file using `fitz.open()`.
    ```python
    doc = fitz.open("/content/EVALUACION FINAL M3.pdf")
    ```
    This line opens the specified PDF file and creates a `Document` object (`doc`) that represents the PDF. This object allows us to access its properties and content.

4.  **Iterating through pages and extracting text:** We loop through each page of the document and extract its text content.
    ```python
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()
    ```
    - `text = ""`: Initializes an empty string variable to store the extracted text from all pages.
    - `for page_num in range(doc.page_count):`: This loop iterates through each page of the document. `doc.page_count` gives the total number of pages in the PDF.
    - `page = doc.load_page(page_num)`: Loads the current page object based on its index (`page_num`).
    - `text += page.get_text()`: Extracts the text content from the current page using the `get_text()` method and appends it to the `text` variable.

5.  **Verifying the extraction (Optional):** We can print a portion of the extracted text to confirm that the process was successful.
    ```python
    print(text[:500]) # Print the first 500 characters to verify
    ```
    This line prints the first 500 characters of the `text` variable, giving us a glimpse of the extracted content to ensure it looks as expected.

This process allows you to programmatically extract the entire text content of a PDF document for further processing or analysis.
